## Web scraping financial data

In [1]:
#install relevant pkgs

import requests
from bs4 import BeautifulSoup
import pandas as pd
import win32com.client as win32

In [2]:
# create a list of tickers of companies our interest

tickers = ['fb', 'tsla', 'aapl', 'amzn']

In [4]:
#take headers to proced web scraping
headers = {'user-agent' : "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_2) AppleWebKit/605.1.15 (KHTML, like Gecko) Safari/605.1.15 Version/13.0.4"}

In [5]:
#save the data in xls spreadsheet
xlApp = win32.Dispatch('Excel.Application')
xlApp.Visible = True
wb = xlApp.Workbooks.Add()

In [6]:
# create a loop to scrap the data from marketwatch.com
    #in this case, we will scrap the relevant information to apply a financial analysis.
for ticker in tickers:
    url = 'https://www.marketwatch.com/investing/stock/{0}/company-profile'.format(ticker)
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    financial_info = {}
    element_tables = soup.select('div [class="element element--table"]')   #here we scrap the info
        
    for x in element_tables:                        
        valuation_type = x.h2.text.strip()          
        df = pd.read_html(str(x))[0]                
        financial_info[valuation_type] = df           
    
    ws = wb.Worksheets.Add()  #exporting the data in a different worksheet
    ws.name = ticker
    
    row_spacing = 2
    
    for table in financial_info.items():
        lastrow = ws.Cells(ws.rows.count, 1).End(-4162).row  
        ws.cells(lastrow+row_spacing, 1).value = table[0]       #header from each table
        ws.cells(lastrow+row_spacing, 1).font.bold =True         #font
        
        ws.Range(
            ws.cells(lastrow+row_spacing+1, 1),                     #adding the data!
            ws.cells(lastrow+table[1].shape[0]+row_spacing, table[1].shape[1])  
        ).value = table[1].values
        
    ws.Rows('1:' + str(row_spacing)).delete
    ws.columns(1).columnwidth = 30
    ws.columns(1).columnwidth = 15  